# fastkmers
*Author: Koki Sasagawa*

`fastkmers.py` will read zipped fastq files directly and find all kmers of size N, then return kmers sorted by their frequencies. 

In [1]:
import fastkmers

### Load sample fq files

For development purposes, the fastq file used for this session has ben downsampled to 1M reads. The original file contained aound ~80 million reads as checked with the following script

```bash
data grep -c "@" sample.fq
```

In [4]:
%%timeit
fastkmers.get_sequences("../data/subset_1M.fq.gz")

Running get_sequences...
Finished in 5.2646s
Running get_sequences...
Finished in 5.2467s
Running get_sequences...
Finished in 5.2262s
Running get_sequences...
Finished in 5.2408s
Running get_sequences...
Finished in 5.2370s
Running get_sequences...
Finished in 5.2477s
Running get_sequences...
Finished in 5.2222s
Running get_sequences...
Finished in 5.2260s
5.27 s ± 9.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


5.27 s on average to load 1 million lines

Estimate the load time for 80 million reads

$$
    80M \times \frac{5.27s}{1M} = 421.6s \times \frac{1min}{60s} = 7min
$$

In [2]:
seq = fastkmers.get_sequences("../data/subset_1M.fq.gz")

Running get_sequences...
Finished in 5.2897s


### Generate kmers

In [7]:
results = fastkmers.sort_results(fastkmers.quantify_kmers(seq, 5))

Running quantify_kmers...
Finished in 311.9757s


With 1M reads, and looking for kmers size 5, it takes around 312 seconds or 5.2 minutes

$$
80M \times \frac{312}{1M} = 24960s
\times \frac{1min}{60s} = 416min
\times \frac{1hr}{60min} = 6.93hrs
$$

With our current algorithm, it takes around 7 hours to process a fullsized fastq file with 80M reads. 
Try multiprocessing on multiple cores to speed up the process

In [17]:
results2 = fastkmers.sort_results(
    fastkmers.merge_kmers(
        fastkmers.parallelize(
            array=seq,
            func=fastkmers.quantify_kmers,
            n_cores=16,
            N=5)
    )
)

Parallelizing quantify_kmers on 16 cores...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Running quantify_kmers...
Finished in 45.3991s
Finished in 45.5584s
Finished in 45.5101s
Finished in 45.5631s
Finished in 45.5978s
Finished in 45.5386s
Finished in 45.4977s
Finished in 45.6534s
Finished in 45.5467s
Finished in 45.6580s
Finished in 45.5748s
Finished in 45.5624s
Finished in 45.5244s
Finished in 45.4913s
Finished in 47.7048s
Finished in 47.4332s


Multiprocessing on 16 cores led to finishing the task in 45~ seconds.

$$
80M \times \frac{45s}{1M} = 3600s
\times \frac{1min}{60s} = 60min
\times \frac{1hr}{60min} = 1hrs
$$

Multiprocessing will enable the task for 80M reads to finish in approximately 1 hour. 
6 hours faster than running the algorithm serially and not parallel.

Get the sequences in a list format

In [22]:
results = [i[0] for i in results]